In [20]:
from multiprocessing import Pool
import os
import time
import utils
import emcee
import numpy as np
import matplotlib.pyplot as plt
import corner
import seaborn as sns
import pickle
import multiprocessing
multiprocessing.set_start_method("fork")

cor, pal = utils.matplotlib_style()

# Set random seed
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# Set number of threads for other non-pool processes to one to avoid conflicts
# with emcee
os.environ["OMP_NUM_THREADS"] = "1"

## MCMC on the two-state mRNA steady state distribution

## Setting the problem  

In this notebook, we will explore trying to infer the parameters of the two-state promoter model using MCMC. The problem can be pose as follows:

Given a set of UMI counts
$$
\underline{u} = [u_1, u_2, \ldots, u_N],
\tag{1}
$$
where $N$ is the number of samples, we want to infer the parameters of the two-state promoter model:
- $k^{(p)}_\text{on}$: The rate at which the promoter transitions from the off
- state to the on
  state.
- $k^{(p)}_\text{off}$: The rate at which the promoter transitions from the on
  state to the off state.
- $r_m$: The mRNA transcription rate.
- $\gamma_m$: The mRNA degradation rate.

One can show that all parameters can be rescaled by the mRNA degradation rate.
Thus, we will perform inference of the following parameters:
$$
\theta = \left\{
    \hat{k}^{(p)}_\text{on}, \hat{k}^{(p)}_\text{off}, \hat{r}_m
    \right\}.
\tag{2}
$$
where $\hat{x} = x/\gamma$.

## Bayesian Inference of the two-state promoter model

With this setup, we can write the inference problem as
$$
\pi{(\theta | \underline{u})} \propto 
\pi{(\underline{u} | \theta)} \pi{(\theta)}.
\tag{3}
$$



### Likelihood

Let's begin by defining the likelihood function. First, we assume that each 
sample $u_i$ is independent and identically distributed. Thus, the likelihood
function can be written as the product
$$
\pi{(\underline{u} | \theta)} = \prod_{i=1}^{N} \pi{(u_i | \theta)}.
\tag{4}
$$
Each term in this product is the probability of observing a given UMI count 
given the parameters of the two-state promoter model. the mRNA count 
distribution from the two-state promoter model is given by
$$
{\scriptstyle
\pi(m \mid \theta) = 
\frac{1}{\Gamma(m + 1)}
\frac{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}} + m
    \right)
}{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}}
    \right)
}
\frac{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}} + \hat{k}^{(p)}_{\text{off}}
    \right)
}{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}} + \hat{k}^{(p)}_{\text{off}} + m
    \right)
}
\left( \hat{r}_m \right)^m \\
\times {}_1F_1 
\left(
    \hat{k}^{(p)}_{\text{on}} + m,
    \hat{k}^{(p)}_{\text{on}} + \hat{k}^{(p)}_{\text{off}} + m,
    - \hat{r}_m
\right),
}
\tag{5}
$$
where ${}_1F_1$ is the Kummer confluent hypergeometric function defined as
$$
{}_1F_1(a, b, z) = \sum_{n=0}^{\infty} \frac{(a)_n}{(b)_n} \frac{z^n}{n!},
\tag{6}
$$
with $(a)_n = a(a+1)\ldots(a+n-1)$ being the Pochhammer symbol or raising
factorial.

It can be shown that if one assumes that for a given mRNA count in one cell,
$m$, the probability of observing each mRNA as a UMI count is a Bernoulli trial,
the probability distribution of the UMI count takes the same functional form 
with only one re-scaled parameter $\hat{r}_u = \hat{r}_m \cdot p$, where $p$ is
the probability of observing a given mRNA as a UMI count. Thus, each of the
terms on the righ-hand side of Eq. 4 can be written as
$$
{\scriptstyle
\pi(u \mid \tilde{\theta}) = 
\frac{1}{\Gamma(u + 1)}
\frac{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}} + u
    \right)
}{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}}
    \right)
}
\frac{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}} + \hat{k}^{(p)}_{\text{off}}
    \right)
}{
    \Gamma
    \left(
        \hat{k}^{(p)}_{\text{on}} + \hat{k}^{(p)}_{\text{off}} + u
    \right)
}
\left( \hat{r}_u \right)^u \\
\times {}_1F_1 
\left(
    \hat{k}^{(p)}_{\text{on}} + u,
    \hat{k}^{(p)}_{\text{on}} + \hat{k}^{(p)}_{\text{off}} + u,
    - \hat{r}_u
\right),
}
\tag{7}
$$
where $\tilde{\theta} = \left\{ \hat{k}^{(p)}_{\text{on}},
\hat{k}^{(p)}_{\text{off}}, \hat{r}_u \right\}$.

We already defined a `python` function to compute this likelihood function that
deals with the numerical challenges of computing the Kummer function. However,
because of the asymptotic limit taken only on certain regimes of the parameter
regimes, this is not amenable to gradient-based MCMC methods. However, we can
still use the not-so-efficient sampler from the `emcee` package. For this, we 
must manually define the log-likelihood function.

In [2]:
def log_likelihood(params, data):
    """
    Compute the log likelihood for the two-state promoter UMI count
    distribution.

    This function calculates the log likelihood of observing the given unique
    molecular identifier (UMI) counts based on the two-state promoter model
    parameters.

    Parameters
    ----------
    params : array-like, shape (3,)
        An array containing the parameters for the steady state UMI
        distribution: - kp_on (float): Rate of activation of the promoter. -
        kp_off (float): Rate of deactivation of the promoter. - ru (float):
        Production rate of the mRNA.
    data : array-like, shape (n, 2)
        An array containing the observed UMI counts and their frequencies: -
        data[:, 0] (int): Unique UMI counts. - data[:, 1] (int): Frequency of
        each UMI count.

    Returns
    -------
    float
        The log likelihood of the observed data given the model parameters.

    Notes
    -----
    The likelihood calculation is optimized by using unique UMI entries and
    their corresponding counts. Instead of computing the probability of each UMI
    count multiple times, the function computes it once for each unique count
    and multiplies the result by the frequency of that count.
    """
    # Unpack parameters
    kp_on, kp_off, ru = params
    # Unpack data
    umi, counts = data[:, 0], data[:, 1]
    return np.sum(
        counts * utils.two_state_log_probability(umi, kp_on, kp_off, ru)
    )

To test this fuction, first we need to generate a synthetic dataset. For this, 
we can use the same `two_state_log_probability` function for some set of parameters. Let's first compute the log probability.

In [ ]:
# Define parameters
k_on = 4.0
k_off = 18.0
r_u = 100.0

# Define range of UMI counts
u_range = np.arange(0, 75)

# Evaluate the log probability
logP = utils.two_state_log_probability(u_range, k_on, k_off, r_u)

# Initialize figure
fig, ax = plt.subplots(1, 1, figsize=(1.75, 1.5))

# Plot the probability
ax.step(u_range, np.exp(logP))

# Label axis
ax.set_xlabel("UMI counts")
ax.set_ylabel("probability")

The probability distribution looks correct. Now, let's generate a synthetic
dataset and test the log-likelihood function.

In [ ]:
np.random.seed(42)
# Define number of samples to draw
n_samples = 10_000

# Draw samples. NOTE: We normalize the probabilities to sum to 1 for the
# function np.random.choice
u_samples = np.random.choice(
    u_range, size=n_samples, p=np.exp(logP) / np.sum(np.exp(logP))
)

# Generate matrix with unique values and their counts needed for the likelihood
# function
data = np.array(np.unique(u_samples, return_counts=True)).T

# Define parameters
k_on = 4.0
k_off = 18.0
r_u = 100.0

# Define range of UMI counts
u_range = np.arange(0, 75)

# Evaluate the log probability
logP = utils.two_state_log_probability(u_range, k_on, k_off, r_u)

# Initialize figure
fig, ax = plt.subplots(1, 1, figsize=(1.75, 1.5))

# Plot the probability
ax.step(u_range, np.exp(logP))
# Histogram the samples
ax.hist(u_samples, bins=u_range, density=True, alpha=0.75)

# Label axis
ax.set_xlabel("UMI counts")
ax.set_ylabel("probability")

Now, let's test the log-likelihood function.

In [ ]:
log_likelihood([k_on, k_off, r_u], data)

Although we have no way of directly assessing the correctness of the
log-likelihood function, we can at least check that the function is not throwing
any errors and is returning a large negative value, as expected. Furthermore,
for the wrong set of parameters, we expect this number to be much more negative.

In [ ]:
log_likelihood([k_on * 2, k_off * 7, r_u * 1/3], data)

### Prior

For our first attempt, we will use naive uniform priors for all parameters
between certain reasonable bounds.

In [39]:
def log_prior(params, pmax):
    """
    Compute the log prior for kinetic parameters in a two-state promoter model.

    This function evaluates the log prior probability of the given kinetic
    parameters based on uniform priors. If any parameter is outside the
    specified bounds, the function returns negative infinity, indicating that
    the parameter set is not allowed.

    Parameters
    ----------
    params : array-like, shape (3,)
        An array containing the kinetic parameters for the steady state UMI
        distribution: - kp_on (float): Rate of activation of the promoter. -
        kp_off (float): Rate of deactivation of the promoter. - ru (float):
        Production rate of the mRNA.
    pmax : array-like, shape (3,)
        An array containing the maximum allowable values for the parameters,
        given in the same order as `params`: - kp_on_max (float): Maximum rate
        of activation of the promoter. - kp_off_max (float): Maximum rate of
        deactivation of the promoter. - ru_max (float): Maximum production rate
        of the mRNA.

    Returns
    -------
    float
        The log prior probability of the parameters. Returns negative infinity
        if any parameter is outside the allowable range, indicating an invalid
        parameter set.
    """
    # Unpack parameters
    kp_on, kp_off, ru = params
    kp_on_max, kp_off_max, ru_max = pmax

    # Define uniform priors for parameters
    if np.any(params < 1e-7):
        return -np.inf
    elif kp_on > kp_on_max or kp_off > kp_off_max or ru > ru_max:
        return -np.inf
    else:
        return 0.0

### Posterior

With likelihood and prior defined, we can now define the posterior distribution
function.

In [40]:
def log_posterior(params, data, pmax, log_sampling=False):
    """
    Compute the log posterior for the two-state promoter model.

    This function calculates the log posterior probability of the given
    parameters based on the observed data, prior distributions, and the
    two-state promoter model. It supports sampling in either linear or log
    scale.

    Parameters
    ----------
    params : array-like, shape (3,)
        An array containing the kinetic parameters for the steady state UMI
        distribution: 
            - kp_on (float): Rate of activation of the promoter. 
            - kp_off (float): Rate of deactivation of the promoter. 
            - ru (float): Production rate of the mRNA.
    data : array-like, shape (n, 2)
        An array containing the observed UMI counts and their frequencies: 
            - data[:, 0] (int): Unique UMI counts. 
            - data[:, 1] (int): Frequency of each UMI count.
    pmax : array-like, shape (3,)
        An array containing the maximum allowable values for the parameters,
        given in the same order as `params`: 
            - kp_on_max (float): Maximum rate of activation of the promoter. 
            - kp_off_max (float): Maximum rate of deactivation of the promoter. 
            - ru_max (float): Maximum production rate of the mRNA.
    log_sampling : bool, optional
        If True, the parameters are assumed to be in log scale and will be
        exponentiated before further calculations. Default is False.

    Returns
    -------
    float
        The log posterior probability of the parameters. Returns negative
        infinity if the parameters are outside the allowable range or if the log
        prior is negative infinity.
    """
    # Boolean logic to sample in linear or in log scale
    if log_sampling:
        params = np.exp(params)

    # Compute log prior
    lp = log_prior(params, pmax)

    # If log prior is -inf, return that
    if lp == -np.inf:
        return -np.inf

    # Compute and return posterior
    return lp + log_likelihood(params, data)

### MCMC sampling

We are ready to perform MCMC sampling. We will use the `emcee` package for this
task. First, we define the sampler with the corresponding parameters.

In [45]:
np.random.seed(42)

# Define the parameters for emcee
n_dim = 3  # number of parameters to fit
n_walkers = 32  # number of walkers
n_burn = 500  # number of burn-in steps
n_steps = 5_000  # number of steps to run the sampler

# Define parameter maximum values
pmax = [20, 100, 200]

# Initialize walkers
p0 = np.zeros([n_walkers, n_dim])
# Initialize kpon
p0[:, 0] = np.random.uniform(1e-5, pmax[0], n_walkers)
# Initialize kpoff
p0[:, 1] = np.random.uniform(1e-5, pmax[1], n_walkers)
# initialize rm
p0[:, 2] = np.random.uniform(1e-5, pmax[2], n_walkers)

Let's now run the sampler. We use the `multiprocessing` backend to speed up the
sampling process.

In [ ]:
np.random.seed(42)

# Defin output file
out_file = f"emcee_chain/two_state_promoter_mcmc_{n_walkers}walkers_{n_steps}steps_{n_burn}burn.pkl"

# Check if file exists
if not os.path.exists(out_file):
    # Run sampling in multiprocessing
    with Pool() as pool:
        # Initialize sampler
        sampler = emcee.EnsembleSampler(
            n_walkers, n_dim, log_posterior, args=(data, pmax), pool=pool
        )
        # Set time at the beginning of the run
        start = time.time()
        # Run the MCMC sampling
        sampler.run_mcmc(p0, n_burn + n_steps, progress=True)
        # Set time at the end of the run
        end = time.time()
        # Quantify the time taken
        multi_time = end - start
        # print the time taken
        print("MCMC ran in {0:.1f} seconds".format(multi_time))
        # Open a file to save the chain
        output = open(out_file, "wb")
        # Dump the chain and the lnprobability to
        pickle.dump(sampler.chain, output)
        pickle.dump(sampler.lnprobability, output)
        # Close the file
        output.close()

Let's load the file.

In [47]:
# Load the flat-chain
with open(out_file, "rb") as file:
    # Open the file
    unpickler = pickle.Unpickler(file)
    # Load the chain
    chain = unpickler.load()
    # Load the lnprobability
    lnprobability = unpickler.load()

# Remove burn-in
chain = chain[:, n_burn:, :]

Let's look at a corner plto of the samples.

In [ ]:
# Initialize subplot
fig, axes = plt.subplots(3, 3, figsize=(3.5, 3.5))

# Draw the corner plot
fig = corner.corner(
    chain,
    bins=50,
    plot_contours=True,
    labels=[r'$k^{(p)}_{on}$', r'$k^{(p)}_{off}$', r'$r_u$'],
    fig=fig
)

# Remove grid lines from all axes
for ax in axes.flatten():
    ax.grid(False)

# Save figure
# plt.savefig(figdir + 'lacUV5_mRNA_prior_corner_plot.pdf', bbox_inches='tight')

The prior bounds here were set artificially low since we knew what the true
values were. However, the important part of this exercise is to see the
unidentifiability of the $k^{(p)}_{\text{off}}$ and $r_u$ parameters. We can see
that these two parameters are perfectly correlated. This is because the model is
degenerate in these two parameters. This is a common problem in the inference of
non-linear models.

## Unidentifiability and the $k^{(p)}_{\text{off}} \gg 1$ limit

The perfect correlation between the $k^{(p)}_{\text{off}}$ and $r_u$ parameters
already indicates that what matter are not the individual values, but their
ratio. One can show that for the limit $k^{(p)}_{\text{off}} \gg 1$, the model
in Eq. 5 converges to a negative binomial distribution of the form
$$
\pi(m \mid \theta) =
{
m + k^{(p)}_{\text{on}} - 1
\choose
m
}
\left( 
    \frac{
        \frac{\hat{r}_u}{\hat{k}^{(p)}_{\text{off}}} 
    }{
        1 + \frac{\hat{r}_u}{\hat{k}^{(p)}_{\text{off}}}
    }
\right)^m
\left(
    \frac{
        1
    }{
        1 + \frac{\hat{r}_u}{\hat{k}^{(p)}_{\text{off}}}
    }
\right)^{k^{(p)}_{\text{on}}},
\tag{8}
$$
where clearly the relevant parameter is the ratio
$\hat{r}_u/\hat{k}^{(p)}_{\text{off}}$.

This implies that the model is not identifiable in this limit. Let's try now to 
define therefore a prior that is more informative about this ratio.

In [50]:
def log_prior_ratio(params, pmax):
    """
    Compute the log prior for kinetic parameters in a two-state promoter model.

    This function evaluates the log prior probability of the given kinetic
    parameters based on uniform priors. If any parameter is outside the
    specified bounds, the function returns negative infinity, indicating that
    the parameter set is not allowed.

    Parameters
    ----------
    params : array-like, shape (3,)
        An array containing the kinetic parameters for the steady state UMI
        distribution: - kp_on (float): Rate of activation of the promoter. - ru
        (float): Production rate of the mRNA. - ru_koff (float): Ratio of
        production rate to the rate of deactivation
          of the promoter.
    pmax : array-like, shape (3,)
        An array containing the maximum allowable values for the parameters,
        given in the same order as `params`: - kp_on_max (float): Maximum rate
        of activation of the promoter. - kp_off_max (float): Maximum rate of
        deactivation of the promoter. - ru_max (float): Maximum production rate
        of the mRNA.

    Returns
    -------
    float
        The log prior probability of the parameters. Returns negative infinity
        if any parameter is outside the allowable range, indicating an invalid
        parameter set.

    Notes
    -----
    The function computes the rate of deactivation (k_off) from the production
    rate (ru) and the ratio (ru_koff). The priors are defined as uniform
    distributions within the specified bounds.
    """
    # Unpack parameters
    kp_on, ru, ru_koff = params
    # Compute k_off from ru and ru_koff
    k_off = ru / ru_koff
    # Unpack pmax
    kp_on_max, kp_off_max, ru_max = pmax

    # Define uniform priors for parameters
    if np.any(params < 1e-7):
        return -np.inf
    elif kp_on > kp_on_max or k_off > kp_off_max or ru > ru_max:
        return -np.inf
    else:
        return 0.0

We must also adapt the likelihood function to take the ratio as a parameter. 

In [51]:
def log_likelihood_ratio(params, data):
    """
    Compute the log likelihood for the two-state promoter UMI count
    distribution.

    This function calculates the log likelihood of observing the given unique
    molecular identifier (UMI) counts based on the two-state promoter model
    parameters.

    Parameters
    ----------
    params : array-like, shape (3,)
        An array containing the parameters for the steady state UMI
        distribution: 
            - kp_on (float): Rate of activation of the promoter. 
            - ru (float): Production rate of the mRNA.
            - ru_koff (float): Ratio of production rate to the rate of
              deactivation of the promoter.
    data : array-like, shape (n, 2)
        An array containing the observed UMI counts and their frequencies: -
        data[:, 0] (int): Unique UMI counts. - data[:, 1] (int): Frequency of
        each UMI count.

    Returns
    -------
    float
        The log likelihood of the observed data given the model parameters.

    Notes
    -----
    The likelihood calculation is optimized by using unique UMI entries and
    their corresponding counts. Instead of computing the probability of each UMI
    count multiple times, the function computes it once for each unique count
    and multiplies the result by the frequency of that count.
    """
    # Unpack parameters
    kp_on, ru, ru_koff = params
    # Compute kp_off from ru and ru_koff
    kp_off = ru / ru_koff
    # Unpack data
    umi, counts = data[:, 0], data[:, 1]
    return np.sum(
        counts * utils.two_state_log_probability(umi, kp_on, kp_off, ru)
    )

And finally the log posterior function.

In [52]:
def log_posterior_ratio(params, data, pmax, log_sampling=False):
    """
    Compute the log posterior for the two-state promoter model.

    This function calculates the log posterior probability of the given
    parameters based on the observed data, prior distributions, and the
    two-state promoter model. It supports sampling in either linear or log
    scale.

    Parameters
    ----------
    params : array-like, shape (3,)
        An array containing the kinetic parameters for the steady state UMI
        distribution: 
            - kp_on (float): Rate of activation of the promoter. 
            - ru (float): Production rate of the mRNA.
            - ru_koff (float): Ratio of production rate to the rate of
    data : array-like, shape (n, 2)
        An array containing the observed UMI counts and their frequencies: 
            - data[:, 0] (int): Unique UMI counts. 
            - data[:, 1] (int): Frequency of each UMI count.
    pmax : array-like, shape (3,)
        An array containing the maximum allowable values for the parameters,
        given in the same order as `params`: 
            - kp_on_max (float): Maximum rate of activation of the promoter. 
            - ru_max (float): Maximum production rate of the mRNA.
            - ru_koff_max (float): Maximum ratio of production rate to the rate
    log_sampling : bool, optional
        If True, the parameters are assumed to be in log scale and will be
        exponentiated before further calculations. Default is False.

    Returns
    -------
    float
        The log posterior probability of the parameters. Returns negative
        infinity if the parameters are outside the allowable range or if the log
        prior is negative infinity.
    """
    # Boolean logic to sample in linear or in log scale
    if log_sampling:
        params = np.exp(params)

    # Compute log prior
    lp = log_prior_ratio(params, pmax)

    # If log prior is -inf, return that
    if lp == -np.inf:
        return -np.inf

    # Compute and return posterior
    return lp + log_likelihood_ratio(params, data)

Let's sample again, this time with the new prior. First, we define the initial
position for the walkers.

In [75]:
np.random.seed(42)

# Define the parameters for emcee
n_dim = 3  # number of parameters to fit
n_walkers = 32  # number of walkers
n_burn = 500  # number of burn-in steps
n_steps = 5_000  # number of steps to run the sampler

# Define parameter maximum values
pmax = [20, 10, 200]

# Initialize walkers
p0 = np.zeros([n_walkers, n_dim])
# Initialize kpon
p0[:, 0] = np.random.uniform(1e-5, pmax[0], n_walkers)
# Initialize kpoff
p0[:, 1] = np.random.uniform(1e-5, pmax[1], n_walkers)
# initialize rm
p0[:, 2] = np.random.uniform(1e-5, pmax[2], n_walkers)

Now, we define the sampler and sample in parallel.

In [76]:
np.random.seed(42)

# Defin output file
out_file = f"emcee_chain/two_state_promoter_ratio_mcmc_{n_walkers}walkers_{n_steps}steps_{n_burn}burn.pkl"

# Check if file exists
if not os.path.exists(out_file):
    # Run sampling in multiprocessing
    with Pool() as pool:
        # Initialize sampler
        sampler = emcee.EnsembleSampler(
            n_walkers, n_dim, log_posterior_ratio, args=(data, pmax), pool=pool
        )
        # Set time at the beginning of the run
        start = time.time()
        # Run the MCMC sampling
        sampler.run_mcmc(p0, n_burn + n_steps, progress=True)
        # Set time at the end of the run
        end = time.time()
        # Quantify the time taken
        multi_time = end - start
        # print the time taken
        print("MCMC ran in {0:.1f} seconds".format(multi_time))
        # Open a file to save the chain
        output = open(out_file, "wb")
        # Dump the chain and the lnprobability to
        pickle.dump(sampler.chain, output)
        pickle.dump(sampler.lnprobability, output)
        # Close the file
        output.close()

Let's load the chain.

In [77]:
# Load the flat-chain
with open(out_file, "rb") as file:
    # Open the file
    unpickler = pickle.Unpickler(file)
    # Load the chain
    chain = unpickler.load()
    # Load the lnprobability
    lnprobability = unpickler.load()

# Remove burn-in
chain = chain[:, n_burn:, :]

Let's look at this corner plot.

In [ ]:
# Initialize subplot
fig, axes = plt.subplots(3, 3, figsize=(3.5, 3.5))

# Draw the corner plot
fig = corner.corner(
    chain,
    bins=50,
    plot_contours=True,
    labels=[r'$k^{(p)}_{on}$', r'$r_u$', r'$r_u/k^{(p)}_{off}$', ],
    fig=fig
)

# Remove grid lines from all axes
for ax in axes.flatten():
    ax.grid(False)